# Scenario 2: A cross-functional team with one data scientist working on a ML model

MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artiacts store: local filesystem

The experiments can be explore locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in the terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [4]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: {mlflow.get_tracking_uri()}")

tracking URI: http://127.0.0.1:5000


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifcats_local/0', creation_time=1761512214444, experiment_id='0', last_update_time=1761512214444, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/26 20:58:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifcats_local/1/d8645ef4af8944cb91a2c292e512dbae/artifacts'
🏃 View run skittish-snake-41 at: http://127.0.0.1:5000/#/experiments/1/runs/d8645ef4af8944cb91a2c292e512dbae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [12]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifcats_local/1', creation_time=1761512284124, experiment_id='1', last_update_time=1761512284124, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifcats_local/0', creation_time=1761512214444, experiment_id='0', last_update_time=1761512214444, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [14]:
client.search_registered_models()

[]

In [25]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

mlflow.register_model(
    model_uri=f"run:/{run_id}/models",
    name="iris-classifier"
)

Successfully registered model 'iris-classifier'.
2025/10/26 21:04:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1761512680330, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761512680330, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='', run_link='', source='run:/d8645ef4af8944cb91a2c292e512dbae/models', status='READY', status_message=None, tags={}, user_id='', version='1'>